# SVM - Kernel

* Potentielle Kernel: RBF, Polynomial, Spectral Mixture Kernel, Sigmoid Kernel
* Calculate SVM using dual decision functions
* Thus: Model theta has as many parameters as examples.
* Kernel function: Measure of similarity between instances.
* Hence: How similar is instance x to each other trainings instance?
* Derivation from the primal into the dual form is necessary (lecture).

Optimization criterion of the dual SVM:

$$
max_{\beta} \sum_{i=1}^n \beta_i - \frac{1}{2} \sum_{i,j=1}^n \beta_i \beta_j y_i y_j k(x_i,x_j) \text{, such that } 0 \leq \beta_i \leq \lambda
$$

* Optimization over parameters beta
* Sparse solution (solution of a problem where most of the elements are zero)
* Reason: Samples only appear as pairwise inner products.
* Sparsity desired property because it often leads to simpler, more interpretable models.
* QPP - Quadratic programming problem

Dual from of the decision function:

$$
f_{\beta}(x)= \sum_{x_i\in SV} \beta_i y_i k(x_i, x)
$$

(SV = Support Vectors)

* Only the support vectors (points with non-zero beta_i) contribute to the decision function.
* Decision function is weighted sum over the support vectors.
* Decides the class based on the sign of this sum.


**Hint:**

This code is a implementation of kernelized empirical risk minimization that aligns with the SVM concepts but uses gradient descent instead of directly solving the dual problem via quadratic programming.